In [ ]:
import pandas as pd
import sqlite3
import datetime
import matplotlib.pyplot as plt
import numpy as np


## Connect to DB, load data and convert to dataframe

In [ ]:
db_path = "data/twitter_data.db"

In [ ]:
conn = sqlite3.connect(db_path)
c = conn.cursor()

In [ ]:
c.execute('SELECT SQLITE_VERSION()')

In [ ]:
c.execute('SELECT * FROM tweets')
raw_data = c.fetchall()

In [ ]:
conn.close()

In [ ]:
raw_df = pd.DataFrame(raw_data)
raw_df.columns = ['text', 'username', 'date', 'location']

### Clean Twitter data

In [ ]:
raw_df['date'] = pd.to_datetime(raw_df['date'])

In [ ]:
import re

def tweet_cleaner(s):    
    s = re.sub(r"http\S+", "", s)
    s = s.replace("’", "'")
    #s = s.replace("'", "")
    s = s.replace("amp", "")
    #s = re.sub(r"\s*[^A-Za-z]+\s*", " ", s)
    #s = s.lower()
    return s

testing = raw_df.text[:]
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))

In [ ]:
# Combine into dataframe
test = pd.DataFrame(test_result)
test.columns = ['text']
clean_df = pd.concat([test, raw_df['date'], raw_df['location']], axis=1)

In [ ]:
clean_df = clean_df[clean_df.text.str.contains('lebron')]

### Tweets per minute

In [ ]:
# time of tweet #edate = datetime.datetime(2018,7,1,23,0,0)

sdate = datetime.datetime(2018,6,30,23,0,0)
edate = datetime.datetime(2018,7,2,23,0,0)
date_df = []
date_df.append(sdate)

date = sdate
while date < edate:
    date += datetime.timedelta(minutes=1)
    date_df.append(date)

In [ ]:
tweet_count = []
for i in range(0,len(date_df)-1):
    mask = (clean_df['date'] <= date_df[i+1]) & (clean_df['date'] > date_df[i])
    tweet_count.append(len(clean_df.loc[mask]))

tweet_count.append(0)

In [ ]:
clean_df['date']

In [ ]:
xtickrange=np.arange(min(x), max(x)+500, 500)
for pos,tick in xtickrange:
    lab[pos]=clean_df['date']

In [ ]:
# Plot tweets per minute
x = range(0,len(tweet_count))
plt.scatter(x, tweet_count,s=1)
plt.xticks((np.arange(min(x), max(x)+500, 500)))
plt.show()

In [ ]:
min(x)